In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [2]:
# Загрузка данных
df = pd.read_csv("train.csv")

In [3]:
# Преобразуем столбец 'Title' удаляя точки после титулов
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')[0]

# Удаление столбца PassengerId
df.drop(columns=['PassengerId'], inplace=True)

df["Sex"] = df["Sex"].map({"male": 0, "female": 1}).astype("int8")

In [4]:

# Вручную заполняем пропущенные значения
df.loc[df['Name'].str.contains('Icard, Miss. Amelie'), 'Embarked'] = 'C'  # Шербур
df.loc[df['Name'].str.contains('Stone, Mrs. George Nelson'), 'Embarked'] = 'S'  # Саутгемптон
# Создаём объект KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Применяем imputer к данным
df[['Age']] = imputer.fit_transform(df[['Age']])
df = pd.get_dummies(df, columns=['Embarked', 'Pclass'], dtype=np.int8)
# Define age group boundaries and labels
age_bins = [0, 1, 5, 10, 14, 18, 30, 50, 70, df['Age'].max()]
age_labels = [
    "Infant", 
    "Toddler", 
    "Child", 
    "Young_Teen", 
    "Teenager", 
    "Young_Adult", 
    "Adult", 
    "Senior", 
    "Elderly"
]

# Create a new column with age groups
df['AgeGroup'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)
df = pd.get_dummies(df, columns=['AgeGroup'], prefix='AgeGroup', dtype=np.int8)
# Создание нового признака FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Признак одиночного пассажира
df['IsAlone'] = (df['FamilySize'] == 1).astype(np.int8)

# Стоимость билета на человека
df['FarePerPerson'] = df['Fare'] / df['FamilySize']
# Извлечение титула (обращения) из имени
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')

# Удаление титула из Name
df['Name'] = df['Name'].str.replace(r' [A-Za-z]+\.', '', regex=True).str.strip()


# Маппинг титулов
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Mlle': 'Miss',
    'Major': 'Rare',
    'Col': 'Rare',
    'Countess': 'Noble',
    'Capt': 'Rare',
    'Ms': 'Miss',
    'Sir': 'Noble',
    'Lady': 'Noble',
    'Mme': 'Mrs',
    'Don': 'Noble',
    'Jonkheer': 'Noble'
}

# Применяем маппинг к титулу в обучающем наборе
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')
df['Title'] = df['Title'].map(title_mapping).fillna('Rare')  # Пропущенные значения заменяем на 'Rare'

# Инициализация LabelEncoder
le = LabelEncoder()

# Обучаем на уникальных титулов, включая 'Rare'
le.fit(df['Title'].unique())

# Преобразуем титулы в числовые значения
df['Title'] = le.transform(df['Title'])

le = LabelEncoder()
df['Name'] = le.fit_transform(df['Name'].str.split(',').str[0])  # Кодируем только фамилию
# Создаем признак количества кают
df['Cabin_Count'] = df['Cabin'].str.split().str.len().fillna(1)
# Разделяем множественные каюты и берем первую
df['Primary_Cabin'] = df['Cabin'].str.split().str[0]
# Разделяем номер основной каюты на букву и число
df['Cabin_Letter'] = df['Primary_Cabin'].str[0]
df['Cabin_Number'] = df['Primary_Cabin'].str.extract('(\d+)').astype(float)
df.filter(like='Cabin', axis=1)

# Удаление столбца Primary_Cabin
df.drop(columns=['Primary_Cabin','Cabin'], inplace=True)
# Разделение данных на train и test (train - с известными Cabin_Letter, test - с NaN)
train_df_Cabin_Letter = df.dropna(subset=['Cabin_Letter'])

# Формирование test_df для строк, где 'Cabin_Letter' содержит NaN
test_df_Cabin_Letter = df[df['Cabin_Letter'].isna()]  # строки с NaN в 'Cabin_Letter'
# Шаг 1: Подготовка данных
X = train_df_Cabin_Letter.drop(columns=['Cabin_Letter'])  # Убираем целевой столбец и 'Cabin'
y = train_df_Cabin_Letter['Cabin_Letter']  # Целевая переменная

# Шаг 3: Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Шаг 4: Обучение модели (например, случайный лес)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Шаг 5: Оценка модели
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Точность модели: {accuracy:.4f}")
# Шаг 1: Подготовка данных для тестовой выборки
X_test_df_Cabin_Letter = test_df_Cabin_Letter.drop(columns=['Cabin_Letter'])  # Убираем целевой столбец и 'Cabin'

# Шаг 2: Предсказания для test_df_Cabin_Letter
predictions = model.predict(X_test_df_Cabin_Letter)

# Шаг 3: Запись предсказаний в основной столбец 'Cabin_Letter'
df.loc[test_df_Cabin_Letter.index, 'Cabin_Letter'] = predictions


# Кодируем титулы с помощью LabelEncoder
le = LabelEncoder()
df['Cabin_Letter'] = le.fit_transform(df['Cabin_Letter'])
# Заполняем пропуски медианой
median_cabin = df['Cabin_Number'].median()
df['Cabin_Number'].fillna(median_cabin, inplace=True)
# Определяем столбцы с бинарными значениями (0 и 1)
binary_columns = [col for col in df.columns if set(df[col].dropna()) <= {0, 1}]

# Столбцы, которые нужно нормализовать (не бинарные)
columns_to_normalize = [col for col in df.columns if col not in binary_columns]

# Инициализация MinMaxScaler для нормализации
scaler = MinMaxScaler()

# Нормализуем только числовые столбцы (не бинарные)
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Разделим данные на признаки (X) и целевую переменную (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация моделей
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(silent=True, random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42),
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1, max_depth=5, num_leaves=31)  # ограничение max_depth и num_leaves
}

# Оценка всех моделей
results = []

for model_name, model in models.items():
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Предсказания на тестовых данных
    y_pred = model.predict(X_test)
    
    # Оценка метрик модели
    accuracy = accuracy_score(y_test, y_pred)
    full_precision = precision_score(y_test, y_pred, average='binary')  # Точность по всем данным
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Сохраняем результаты
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Full Precision': full_precision,
        'Recall': recall,
        'F1 Score': f1
    })

# Преобразуем результаты в DataFrame
results_df = pd.DataFrame(results)

# Выводим таблицу с результатами
print(results_df)

# Сохраняем лучшую модель (по Full Precision или другой метрике, если нужно)
best_model_name = results_df.loc[results_df['Accuracy'].idxmax(), 'Model']
best_model = models[best_model_name]

# Сохраняем модель
joblib.dump(best_model, f"{best_model_name}_model.pkl")

print(f"Лучшая модель: {best_model_name}")


<>:84: SyntaxWarning: invalid escape sequence '\d'
<>:84: SyntaxWarning: invalid escape sequence '\d'
C:\Users\14488\AppData\Local\Temp\ipykernel_11928\362483581.py:84: SyntaxWarning: invalid escape sequence '\d'
  df['Cabin_Number'] = df['Primary_Cabin'].str.extract('(\d+)').astype(float)


ValueError: could not convert string to float: 'PC 17610'

In [2]:
# Загрузка тестовых данных для предсказания
df = pd.read_csv("test.csv")

In [3]:
# Преобразуем столбец 'Title' удаляя точки после титулов
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')[0]

In [ ]:
df[['Pclass', 'SibSp', 'Parch']] = df[['Pclass', 'SibSp', 'Parch']].astype('int8')

# Удаление столбца PassengerId
df.drop(columns=['Ticket'], inplace=True)
df["Sex"] = df["Sex"].map({"male": 0, "female": 1}).astype("int8")
# Вручную заполняем пропущенные значения
df.loc[df['Name'].str.contains('Icard, Miss. Amelie'), 'Embarked'] = 'C'  # Шербур
df.loc[df['Name'].str.contains('Stone, Mrs. George Nelson'), 'Embarked'] = 'S'  # Саутгемптон
# Создаём объект KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Применяем imputer к данным
df[['Age']] = imputer.fit_transform(df[['Age']])
df = pd.get_dummies(df, columns=['Embarked', 'Pclass'], dtype=np.int8)
# Define age group boundaries and labels
age_bins = [0, 1, 5, 10, 14, 18, 30, 50, 70, df['Age'].max()]
age_labels = [
    "Infant", 
    "Toddler", 
    "Child", 
    "Young_Teen", 
    "Teenager", 
    "Young_Adult", 
    "Adult", 
    "Senior", 
    "Elderly"
]

# Create a new column with age groups
df['AgeGroup'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)
df = pd.get_dummies(df, columns=['AgeGroup'], prefix='AgeGroup', dtype=np.int8)
# Создание нового признака FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Признак одиночного пассажира
df['IsAlone'] = (df['FamilySize'] == 1).astype(np.int8)

# Стоимость билета на человека
df['FarePerPerson'] = df['Fare'] / df['FamilySize']
# Извлечение титула (обращения) из имени
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')

# Удаление титула из Name
df['Name'] = df['Name'].str.replace(r' [A-Za-z]+\.', '', regex=True).str.strip()


# Маппинг титулов
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Mlle': 'Miss',
    'Major': 'Rare',
    'Col': 'Rare',
    'Countess': 'Noble',
    'Capt': 'Rare',
    'Ms': 'Miss',
    'Sir': 'Noble',
    'Lady': 'Noble',
    'Mme': 'Mrs',
    'Don': 'Noble',
    'Jonkheer': 'Noble'
}

# Применяем маппинг к титулу в обучающем наборе
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')
df['Title'] = df['Title'].map(title_mapping).fillna('Rare')  # Пропущенные значения заменяем на 'Rare'

# Инициализация LabelEncoder
le = LabelEncoder()

# Обучаем на уникальных титулов, включая 'Rare'
le.fit(df['Title'].unique())

# Преобразуем титулы в числовые значения
df['Title'] = le.transform(df['Title'])

le = LabelEncoder()
df['Name'] = le.fit_transform(df['Name'].str.split(',').str[0])  # Кодируем только фамилию
# Создаем признак количества кают
df['Cabin_Count'] = df['Cabin'].str.split().str.len().fillna(1)
# Разделяем множественные каюты и берем первую
df['Primary_Cabin'] = df['Cabin'].str.split().str[0]
# Разделяем номер основной каюты на букву и число
df['Cabin_Letter'] = df['Primary_Cabin'].str[0]
df['Cabin_Number'] = df['Primary_Cabin'].str.extract('(\d+)').astype(float)
df.filter(like='Cabin', axis=1)

# Удаление столбца Primary_Cabin
df.drop(columns=['Primary_Cabin','Cabin'], inplace=True)

In [ ]:
# Разделение данных на train и test (train - с известными Cabin_Letter, test - с NaN)
train_df_Cabin_Letter = df.dropna(subset=['Cabin_Letter'])

# Формирование test_df для строк, где 'Cabin_Letter' содержит NaN
test_df_Cabin_Letter = df[df['Cabin_Letter'].isna()]  # строки с NaN в 'Cabin_Letter'
# Шаг 1: Подготовка данных
X = train_df_Cabin_Letter.drop(columns=['Cabin_Letter'])  # Убираем целевой столбец и 'Cabin'
y = train_df_Cabin_Letter['Cabin_Letter']  # Целевая переменная

# Шаг 3: Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Шаг 4: Обучение модели (например, случайный лес)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Шаг 5: Оценка модели
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Точность модели: {accuracy:.4f}")
# Шаг 1: Подготовка данных для тестовой выборки
X_test_df_Cabin_Letter = test_df_Cabin_Letter.drop(columns=['Cabin_Letter'])  # Убираем целевой столбец и 'Cabin'

# Шаг 2: Предсказания для test_df_Cabin_Letter
predictions = model.predict(X_test_df_Cabin_Letter)

# Шаг 3: Запись предсказаний в основной столбец 'Cabin_Letter'
df.loc[test_df_Cabin_Letter.index, 'Cabin_Letter'] = predictions


# Кодируем титулы с помощью LabelEncoder
le = LabelEncoder()
df['Cabin_Letter'] = le.fit_transform(df['Cabin_Letter'])
# Заполняем пропуски медианой
median_cabin = df['Cabin_Number'].median()
df['Cabin_Number'].fillna(median_cabin, inplace=True)
# Определяем столбцы с бинарными значениями (0 и 1)
binary_columns = [col for col in df.columns if set(df[col].dropna()) <= {0, 1}]

# Столбцы, которые нужно нормализовать (не бинарные), исключая 'PassengerId'
columns_to_normalize = [col for col in df.columns if col not in binary_columns and col != 'PassengerId']


# Инициализация MinMaxScaler для нормализации
scaler = MinMaxScaler()

# Нормализуем только числовые столбцы (не бинарные)
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [ ]:
# Проверить наличие пропущенных значений в DataFrame
nan_check = df.isna().sum()

# Выводим количество пропущенных значений по каждому столбцу
print(nan_check)
# Заполняем пропущенные значения медианой по каждому столбцу
df = df.apply(lambda col: col.fillna(col.median()) if col.isna().sum() > 0 else col)


In [7]:
# Прогнозируем выживаемость
X_test = df.drop('PassengerId', axis=1)
y_pred_test = best_model.predict(X_test)

In [8]:
# Сохраняем результат
submission = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': y_pred_test})
submission.to_csv('gender_submission.csv', index=False)